In [7]:
import argparse
import logging
import os
import pickle
from pathlib import Path
from typing import Any, Dict, List, Tuple, Union, Optional
import json

import numpy as np
import pandas as pd
from s1isp.decoder import (
    EUdfDecodingMode, 
    SubCommutatedDataDecoder,
    decode_stream, 
    decoded_stream_to_dict, 
    decoded_subcomm_to_dict
)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)    



def decode_radar_file(input_file: Union[str, Path], apply_transformations: bool = False) -> Tuple[List[Dict[str, Any]], List[int]]:
    """Decode Sentinel-1 Level 0 radar file and extract burst data with ephemeris.

    This function performs complete decoding of a radar file, extracting both
    the radar echo data and associated metadata including ephemeris information.

    Args:
        input_file (Union[str, Path]): Path to the input Level 0 radar file.
        apply_transformations (bool): Whether to apply parameter transformations to convert raw values to physical units.

    Returns:
        Dict

    Raises:
        FileNotFoundError: If input file does not exist.
        RuntimeError: If decoding process fails.
    """
    input_file = Path(input_file)
    if not input_file.exists():
        raise FileNotFoundError(f'Input file not found: {input_file}')
    
    logger.info(f'Starting decode process for: {input_file}')
    
    try:
        # Decode the stream with UDF data
        records, _, subcom_data_records = decode_stream(
            str(input_file),
            udf_decoding_mode=EUdfDecodingMode.DECODE,
        )
        
        assert records, 'Records list cannot be empty'
        logger.info(f'Decoded {len(records)} records from file')
        
        # A) EPHEMERIS EXTRACTION:
        if subcom_data_records:
            subcom_decoder = SubCommutatedDataDecoder()
            subcom_decoded = subcom_decoder.decode(subcom_data_records)
            subcom_dict = decoded_subcomm_to_dict(subcom_decoded=subcom_decoded)
            ephemeris_df = pd.DataFrame(subcom_dict)
            logger.info(f'Extracted ephemeris data with {len(ephemeris_df)} records')
        else:
            logger.warning('No subcommutated data found, creating empty ephemeris DataFrame')
            ephemeris_df = pd.DataFrame()
        
    
        # B) RADAR DATA & META
        signal_types = {'noise': 1, 'tx_cal': 8, 'echo': 0}
        echo_signal_type = signal_types['echo']
        filtered_records = [ # Filter echo records
            record for record in records 
            if record[1].radar_configuration_support.ses.signal_type == echo_signal_type
        ]
        
        # METADATA
        headers_data = decoded_stream_to_dict(filtered_records, enum_value=True)
        burst_metadata = pd.DataFrame(headers_data)
        
        # RADAR ECHOES
        udf_data = [record.udf for record in filtered_records]
        
        if udf_data:
            # Check if all arrays have the same length
            lengths = [len(x) for x in udf_data]
            if len(set(lengths)) == 1:
                # All same length - can stack directly
                radar_data = np.column_stack(udf_data)
            else:
                # Variable lengths - pad with zeros to max length
                max_len = max(lengths)
                radar_data = np.column_stack([
                    np.pad(arr, (0, max_len - len(arr)), mode='constant', constant_values=0)
                    for arr in udf_data
                ])
        else:
            radar_data = np.array([])
        
        burst_dict = {
            'echo': radar_data,
            'metadata': burst_metadata,
            'ephemeris': ephemeris_df
        }
        
        return burst_dict
        
    except Exception as e:
        logger.error(f'Decoding failed for {input_file}: {e}')
        raise RuntimeError(f'File decoding failed: {e}') from e


In [8]:
out = decode_radar_file(
'/Data_large/marine/PythonProjects/SAR/sarpyx/data/maya4ps/S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE/s1a-s1-raw-s-vh-20240629t123133-20240629t123159-054538-06a32d.dat'
)

INFO:__main__:Starting decode process for: /Data_large/marine/PythonProjects/SAR/sarpyx/data/maya4ps/S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE/s1a-s1-raw-s-vh-20240629t123133-20240629t123159-054538-06a32d.dat
decoded: 47591 packets [02:34, 308.91 packets/s] 
INFO:__main__:Decoded 47591 records from file
decoded: 47591 packets [02:34, 308.91 packets/s] 
INFO:__main__:Decoded 47591 records from file
INFO:s1isp.decoder.SubCommutatedDataDecoder:763 sub-commutated data cycles collected.
INFO:s1isp.decoder.SubCommutatedDataDecoder:763 sub-commutated data cycles collected.
INFO:s1isp.decoder.SubCommutatedDataDecoder:26 incomplete sub-commutated data cycles.
INFO:s1isp.decoder.SubCommutatedDataDecoder:26 incomplete sub-commutated data cycles.
INFO:__main__:Extracted ephemeris data with 737 records
INFO:__main__:Extracted ephemeris data with 737 records
